In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import re
from jupyter_dash import JupyterDash  # Use JupyterDash instead of dash.Dash
import dash
from dash import html
import dash_cytoscape as cyto

In [2]:
file_path = 'D:\\test\\ExcelManualCleaning.csv'
df = pd.read_csv(file_path) 

In [3]:
df.head()

,s_no,acknowledgement_no,transaction_id,layer,to_account_no,action_taken_by_bank_wallet_pgpa_merchant_insurance,bank_wallet_pgpa_merchant__insurance,from_account_no,ifsc_code,cheque_no,...,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38
0,1,`30209240022092,`PUNBR52024090612314232,2,`8830210000008880,Money Transfer to,Federal Bank,`24610100000643,FDRL0001010,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,`30209240022092,`PUNBR52024090612314232,2,`8830210000008880,Money Transfer to,Canara Bank,`110196345628,CNRB0004785,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,`30209240022092,`PUNBR52024090612314232,2,`8830210000008880,Money Transfer to,Canara Bank,`110189607716,CNRB0004785,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,`30209240022092,`PUNBR52024090612314232,2,`8830210000008880,Money Transfer to,South Indian Bank,`179053000017033,SIBL0000848,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,`30209240022092,`PUNBR52024090612314232,2,`8830210000008880,Money Transfer to,Bandhan Bank,`20200057515505,BDBL0001815,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df['amount']=df['amount'].astype(int)

In [5]:
df['layer'] = df['layer'].astype(int)

In [6]:
df = df.drop(['s_no'],axis=1)
# Remove text within square brackets
df['to_account_no'] = df['to_account_no'].str.replace(r'\s*\[.*?\]', '', regex=True)

In [7]:
# Define the columns to clean
columns_to_clean = [
    'acknowledgement_no', 'transaction_id', 'layer',
    'from_account_no', 'utr_number', 'amount','to_account_no'
]

# Function to clean data in a column
def clean_column_data(series):
    # Remove special characters and trim spaces
    series = series.apply(lambda x: re.sub(r'[^\w\s]', '', str(x)).strip() if pd.notna(x) else x)
    return series

# Apply the cleaning function to each specified column
for col in columns_to_clean:
    if col in df.columns:
        df[col] = clean_column_data(df[col])

# Display the cleaned data to verify
print(df[columns_to_clean].head())

  acknowledgement_no          transaction_id layer  from_account_no  \
0     30209240022092  PUNBR52024090612314232     2   24610100000643   
1     30209240022092  PUNBR52024090612314232     2     110196345628   
2     30209240022092  PUNBR52024090612314232     2     110189607716   
3     30209240022092  PUNBR52024090612314232     2  179053000017033   
4     30209240022092  PUNBR52024090612314232     2   20200057515505   

               utr_number  amount     to_account_no  
0  DBSSR52024090605165954  270000  8830210000008880  
1  DBSSR52024090605165914  200000  8830210000008880  
2             CNRB0004785  399999  8830210000008880  
3  DBSSR52024090605165915  371900  8830210000008880  
4  DBSSR52024090605165951  290178  8830210000008880  


In [8]:
df['unique_id'] = pd.Series(range(1, len(df) + 1))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 39 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   acknowledgement_no                                   156 non-null    object 
 1   transaction_id                                       156 non-null    object 
 2   layer                                                156 non-null    object 
 3   to_account_no                                        156 non-null    object 
 4   action_taken_by_bank_wallet_pgpa_merchant_insurance  156 non-null    object 
 5   bank_wallet_pgpa_merchant__insurance                 156 non-null    object 
 6   from_account_no                                      156 non-null    object 
 7   ifsc_code                                            96 non-null     object 
 8   cheque_no                                            156 non-null    i

In [9]:
#df.isnull().sum() + (df == '').sum()
# Define the columns you want to update
columns_to_update = ['to_account_no', 'from_account_no', 'ifsc_code', 'utr_number']

# Replace empty strings and null values with 'NaaN'
df[columns_to_update] = df[columns_to_update].replace('', 'NaaN').fillna('NaaN')
df.isnull().sum() + (df == '').sum()

acknowledgement_no                                       0
transaction_id                                           0
layer                                                    0
to_account_no                                            0
action_taken_by_bank_wallet_pgpa_merchant_insurance      0
bank_wallet_pgpa_merchant__insurance                     0
from_account_no                                          0
ifsc_code                                                0
cheque_no                                                0
mid                                                    151
tid                                                    151
approval_code                                          151
merchant_name                                          151
transaction_date                                         0
utr_number                                               0
amount                                                   0
reference_no                                           1

In [10]:
df['layer'] = pd.to_numeric(df['layer'], errors='coerce')
print(df['layer'].unique())

[2 3 1 4 5 6 7]


In [11]:
layers = df.groupby('layer')
layer1 = layers.get_group(1)
layer1

,acknowledgement_no,transaction_id,layer,to_account_no,action_taken_by_bank_wallet_pgpa_merchant_insurance,bank_wallet_pgpa_merchant__insurance,from_account_no,ifsc_code,cheque_no,mid,...,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,unique_id
21,30209240022092,SBINR52024090646591003,1,8830210000008880,Transaction put on hold,DBS Bank,1146110010053650,NaaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
34,30209240022092,PUNBR52024090612314232,1,10832030036580,Money Transfer to,DBS Bank,8830210000008880,DBSS0IN0830,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35
38,30209240022092,PUNBR52024090612314232,1,8830210000008880,Other,DBS Bank,NaaN,NaaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39
39,30209240022092,SBINR52024090646591003,1,10047593897,Money Transfer to,DBS Bank,8830210000008880,DBSS0IN0830,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
40,30209240022092,965562,1,10047593897,Money Transfer to,State Bank of India,43017388358,SBIN0001464,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41
41,30209240022092,965563,1,10047593897,Money Transfer to,State Bank of India,43280321955,SBIN0011519,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42
42,30209240022092,965562,1,43017388358,Other,State Bank of India,NaaN,NaaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43
43,30209240022092,965563,1,43280321955,Other,State Bank of India,NaaN,NaaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44
44,30209240022092,10047593897,1,43017388358,Other,State Bank of India,NaaN,NaaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45
45,30209240022092,10047593897,1,43280321955,Other,State Bank of India,NaaN,NaaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46


In [12]:
# Assuming df contains your data
layers = df.groupby('layer')
for layer_num, layer_data in layers:
    # Get distinct account numbers
    print(layer_data)
    distinct_accounts = layer_data['from_account_no'].unique()
    print(distinct_accounts)

   acknowledgement_no          transaction_id  layer     to_account_no  \
21     30209240022092  SBINR52024090646591003      1  8830210000008880   
34     30209240022092  PUNBR52024090612314232      1    10832030036580   
38     30209240022092  PUNBR52024090612314232      1  8830210000008880   
39     30209240022092  SBINR52024090646591003      1       10047593897   
40     30209240022092                  965562      1       10047593897   
41     30209240022092                  965563      1       10047593897   
42     30209240022092                  965562      1       43017388358   
43     30209240022092                  965563      1       43280321955   
44     30209240022092             10047593897      1       43017388358   
45     30209240022092             10047593897      1       43280321955   
52     30209240022092  SBINR52024090646591003      1  8830210000008880   

   action_taken_by_bank_wallet_pgpa_merchant_insurance  \
21                            Transaction put on hold

In [13]:
def format_amount_indian(amount):
    # Convert the amount to a string and remove any existing commas
    amount_str = str(amount).replace(',', '')
    
    # Check if the number has more than 3 digits
    if len(amount_str) > 3:
        # Get the last 3 digits
        last_three = amount_str[-3:]
        # Get the remaining digits
        remaining = amount_str[:-3]
        # Group digits in thousands (group of 2 after the first group of 3)
        grouped = [remaining[max(0, i-2):i] for i in range(len(remaining), 0, -2)]
        # Reverse and join the grouped digits with commas
        formatted_remaining = ','.join(grouped[::-1])
        # Concatenate the formatted remaining part with the last three digits
        formatted_amount = f'{formatted_remaining},{last_three}'
    else:
        # For amounts less than or equal to 999, no formatting is needed
        formatted_amount = amount_str

    return formatted_amount

In [14]:
ackno = df['acknowledgement_no'].unique()
ackname = 'Transaction Graph For (Ack No): ' + str(ackno)
ackname

"Transaction Graph For (Ack No): ['30209240022092']"

In [82]:
#layer1_records = df[(df['layer'] == 1) & (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
#layer1_nodes = layer1_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date','ifsc_code', 'amount', 'remarks']]

#layer1_nodes

In [24]:
from graphviz import Digraph

# Create a directed graph using Graphviz with hierarchical layout
dot = Digraph()

# Set the graph layout to be hierarchical (left-right) and use L-shaped edges
dot.attr(rankdir='LR', splines='ortho')

# Get the distinct unique layers from the dataset
layers = sorted(df['layer'].unique())

# Initialize the previous layer nodes (starting with an empty set)
previous_layer_nodes = None

# Set to track already added edges to avoid duplicates
added_edges = set()

# Loop through each distinct layer in the dataset
for idx, layer in enumerate(layers):
    
    # Filter the records for the current layer
    layer_records = df[df['layer'] == layer]

    # Select relevant columns from the filtered dataset
    layer_nodes = layer_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'ifsc_code', 'amount', 'remarks', 'action_taken_by_bank_wallet_pgpa_merchant_insurance']]
    
    # Add nodes for the current layer (from_account_no as nodes, with rectangular shape)
    for _, row in layer_nodes.iterrows():
        # Default color is black
        node_color = 'black'
        
        # Change node color if 'WITHDRAWAL' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
        if 'WITHDRAWAL' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
            node_color = '#0000ff'  # Blue color for withdrawal
            
        # Change node color if 'ON HOLD' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
        elif 'ON HOLD' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
            node_color = '#ff0000'  # Red color for on hold
            
        # Add node with the specific color and rectangular shape
        dot.node(
            str(row['from_account_no']), 
            label=f"Acc: {row['from_account_no']}\nTransID: {row['transaction_id']}\nIFSC Code: {row['ifsc_code']}\nAmount: {format_amount_indian(row['amount'])}\n{row['transaction_date']}\n{row['action_taken_by_bank_wallet_pgpa_merchant_insurance']}", 
            shape='box',
            color=node_color
        )

        # Check for NaN in from_account_no
        if str(row['from_account_no']).upper() == "NAAN":
            # Create a unique identifier for each NaN node based on its transaction details
            nan_node_id = f"NaaN_{row['transaction_id']}"  # Unique ID for this specific NaaN transaction
            node_color = '#cccccc'
            if 'WITHDRAWAL' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
                node_color = '#0000ff'  # Blue color for withdrawal
            
            # Change node color if 'ON HOLD' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
            elif 'ON HOLD' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
                node_color = '#ff0000'  # Red color for on hold
            dot.node(nan_node_id, label=f"From: NaaN\nTransID: {row['transaction_id']}\nIFSC Code: {row['ifsc_code']}\nAmount: {format_amount_indian(row['amount'])}\n{row['transaction_date']}\n{row['action_taken_by_bank_wallet_pgpa_merchant_insurance']}", shape='ellipse', color=node_color)

            # Add an edge from the current node to the unique NaaN node
            edge = (str(row['to_account_no']), nan_node_id)
            if edge not in added_edges:
                dot.edge(str(row['to_account_no']), nan_node_id)
                added_edges.add(edge)

    # If this is not the first layer, add edges between the previous layer and the current one
    if previous_layer_nodes is not None:
        # Add edges based on "to_account_no" in the current layer matching "from_account_no" in the previous layer
        for _, row in layer_nodes.iterrows():
            matching_previous_layer_nodes = previous_layer_nodes[previous_layer_nodes['from_account_no'] == row['to_account_no']]
            
            for _, prev_row in matching_previous_layer_nodes.iterrows():
                # Create an identifier for the edge to check for duplicates
                edge = (str(prev_row['from_account_no']), str(row['from_account_no']))
                
                # Add edge if it hasn't been added yet
                if edge not in added_edges:
                    dot.edge(str(prev_row['from_account_no']), str(row['from_account_no']))
                    added_edges.add(edge)  # Track the added edge

    # Set the current layer as the previous layer for the next iteration
    previous_layer_nodes = layer_nodes

# Save and view the graph (optional)
dot.render('dfs_transaction_graph_with_individual_nan', format='png', cleanup=True)
dot.view()


In [83]:
from graphviz import Digraph
import pandas as pd  # Assuming pandas is needed

# Create a directed graph using Graphviz with hierarchical layout
dot = Digraph()

# Set the graph layout to be hierarchical (left-right) and use L-shaped edges
dot.attr(rankdir='LR', splines='ortho')

# Get the distinct unique layers from the dataset
layers = sorted(df['layer'].unique())

# Initialize the previous layer nodes (starting with an empty set)
previous_layer_nodes = None

# Set to track already added edges and nodes to avoid duplicates
added_edges = set()
added_nodes = set()

# Loop through each distinct layer in the dataset
for idx, layer in enumerate(layers):
    layer_records = df[df['layer'] == layer]

    # Select relevant columns from the filtered dataset
    layer_nodes = layer_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'ifsc_code', 'amount', 'remarks', 'action_taken_by_bank_wallet_pgpa_merchant_insurance']]
    
    # Add nodes for the current layer (from_account_no as nodes, with rectangular shape)
    for _, row in layer_nodes.iterrows():
        node_color = 'black'  # Default color

        # Get the action field and ensure it's a string
        action_value = str(row['action_taken_by_bank_wallet_pgpa_merchant_insurance']).upper()
        
        # Color coding based on action taken
        if 'WITHDRAWAL' in action_value:
            node_color = '#0000ff'  # Blue color for withdrawal
        elif 'ON HOLD' in action_value:
            node_color = '#ff0000'  # Red color for on hold
            
        # Add node with the specific color and rectangular shape, avoiding duplicates
        from_account = str(row['from_account_no'])
        if from_account not in added_nodes:
            dot.node(
                from_account, 
                label=f"Acc: {from_account}\nTransID: {row['transaction_id']}\nIFSC Code: {row['ifsc_code']}\nAmount: {format_amount_indian(row['amount'])}\n{row['transaction_date']}\n{row['action_taken_by_bank_wallet_pgpa_merchant_insurance']}", 
                shape='box',
                color=node_color
            )
            added_nodes.add(from_account)

    # If this is not the first layer, add edges between the previous layer and the current one
    if previous_layer_nodes is not None:
        for _, row in layer_nodes.iterrows():
            to_account_no = str(row['to_account_no']) if pd.notna(row['to_account_no']) else 'naan'
            
            # Create the "naan" node if it doesn't exist
            if to_account_no == 'naan' and to_account_no not in added_nodes:
                dot.node(
                    to_account_no, 
                    label=f"Acc: naan\nPlaceholder Account", 
                    shape='box',
                    color='grey'
                )
                added_nodes.add(to_account_no)

            # Add an edge between "from_account_no" and "to_account_no", avoiding duplicates
            edge = (str(row['from_account_no']), to_account_no)
            if edge not in added_edges:
                dot.edge(str(row['from_account_no']), to_account_no)
                added_edges.add(edge)

    # Set the current layer as the previous layer for the next iteration
    previous_layer_nodes = layer_nodes

# Save and view the graph (optional)
dot.render('dfs_transaction_graph1', format='png', cleanup=True)
dot.view()


'dfs_transaction_graph1.pdf'

In [66]:
from graphviz import Digraph

# Create a directed graph using Graphviz with hierarchical layout
dot = Digraph()

# Set the graph layout to be hierarchical (left-right) and use L-shaped edges
dot.attr(rankdir='LR', splines='ortho')

# Get the distinct unique layers from the dataset
layers = sorted(df['layer'].unique())

# Initialize the previous layer nodes (starting with an empty set)
previous_layer_nodes = None

# Set to track already added edges to avoid duplicates
added_edges = set()

# Loop through each distinct layer in the dataset
for idx, layer in enumerate(layers):
    layer_records = df[df['layer'] == layer]

    # Select relevant columns from the filtered dataset
    layer_nodes = layer_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'ifsc_code', 'amount', 'remarks', 'action_taken_by_bank_wallet_pgpa_merchant_insurance']]
    
    # Add nodes for the current layer (from_account_no as nodes, with rectangular shape)
    for _, row in layer_nodes.iterrows():
        # Default color is black
        node_color = 'black'
        
        # Change node color if 'WITHDRAWAL' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
        if 'WITHDRAWAL' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
            node_color = '#0000ff'  # Blue color for withdrawal
            
        # Change node color if 'ON HOLD' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
        elif 'ON HOLD' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
            node_color = '#ff0000'  # Red color for on hold
            
        # Add node with the specific color and rectangular shape
        dot.node(
            str(row['from_account_no']), 
            label=f"Acc: {row['from_account_no']}\nTransID: {row['transaction_id']}\nIFSC Code: {row['ifsc_code']}\nAmount: {format_amount_indian(row['amount'])}\n{row['transaction_date']}\n{row['action_taken_by_bank_wallet_pgpa_merchant_insurance']}", 
            shape='box',
            color=node_color
        )

    # If this is not the first layer, add edges between the previous layer and the current one
    if previous_layer_nodes is not None:
        # Add edges based on "to_account_no" in the current layer matching "from_account_no" in the previous layer
        for _, row in layer_nodes.iterrows():
            # Check if "to_account_no" is not null or empty
            if pd.notna(row['to_account_no']) and row['to_account_no'] != '':
                to_account_no = str(row['to_account_no'])
            else:
                # If "to_account_no" is null or empty, treat it as 'naan'
                to_account_no = 'naan'
                
                # Create the "naan" node if it doesn't exist
                dot.node(
                    to_account_no, 
                    label=f"Acc: naan\nPlaceholder Account", 
                    shape='box',
                    color='grey'  # Grey to indicate a placeholder node
                )

            # Add an edge between "from_account_no" and the corresponding "to_account_no"
            edge = (str(row['from_account_no']), to_account_no)
            if edge not in added_edges:
                dot.edge(str(row['from_account_no']), to_account_no)
                added_edges.add(edge)


    # Set the current layer as the previous layer for the next iteration
    previous_layer_nodes = layer_nodes

# Save and view the graph (optional)
dot.render('dfs_transaction_graph1', format='png', cleanup=True)
dot.view()

'dfs_transaction_graph1.pdf'

In [15]:
from graphviz import Digraph

# Create a directed graph using Graphviz with hierarchical layout
dot = Digraph()

# Set the graph layout to be hierarchical (left-right) and use L-shaped edges
dot.attr(rankdir='LR', splines='ortho')

# Get the distinct unique layers from the dataset
layers = sorted(df['layer'].unique())

# Initialize the previous layer nodes (starting with an empty set)
previous_layer_nodes = None

# Set to track already added edges to avoid duplicates
added_edges = set()

# Loop through each distinct layer in the dataset
for idx, layer in enumerate(layers):
    
    # Step 1: For the first layer, apply Money Transfer filter
    #if idx == 0:
        #layer_records = df[(df['layer'] == layer) & (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
    # Step 2: For other layers, only filter by layer
    #else:
    layer_records = df[df['layer'] == layer]

    # Select relevant columns from the filtered dataset
    layer_nodes = layer_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'ifsc_code', 'amount', 'remarks', 'action_taken_by_bank_wallet_pgpa_merchant_insurance']]
    
    # Add nodes for the current layer (from_account_no as nodes, with rectangular shape)
    for _, row in layer_nodes.iterrows():
        # Default color is black
        node_color = 'black'
        
        # Change node color if 'WITHDRAWAL' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
        if 'WITHDRAWAL' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
            node_color = '#0000ff'  # Blue color for withdrawal
            
        # Change node color if 'ON HOLD' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
        elif 'ON HOLD' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
            node_color = '#ff0000'  # Red color for on hold
            
        # Add node with the specific color and rectangular shape
        dot.node(
            str(row['from_account_no']), 
            label=f"Acc: {row['from_account_no']}\nTransID: {row['transaction_id']}\nIFSC Code: {row['ifsc_code']}\nAmount: {format_amount_indian(row['amount'])}\n{row['transaction_date']}\n{row['action_taken_by_bank_wallet_pgpa_merchant_insurance']}", 
            shape='box',
            color=node_color
        )

    # If this is not the first layer, add edges between the previous layer and the current one
    if previous_layer_nodes is not None:
        # Add edges based on "to_account_no" in the current layer matching "from_account_no" in the previous layer
        for _, row in layer_nodes.iterrows():
            matching_previous_layer_nodes = previous_layer_nodes[previous_layer_nodes['from_account_no'] == row['to_account_no']]
            
            for _, prev_row in matching_previous_layer_nodes.iterrows():
                # Create an identifier for the edge to check for duplicates
                edge = (str(prev_row['from_account_no']), str(row['from_account_no']))
                
                # Add edge if it hasn't been added yet
                if edge not in added_edges:
                    # Add L-shaped edge from the previous layer's from_account_no to the current layer's from_account_no
                    # dot.edge(str(prev_row['from_account_no']), str(row['from_account_no']), label=f"Transfer on {row['transaction_date']}")
                    dot.edge(str(prev_row['from_account_no']), str(row['from_account_no']))
                    added_edges.add(edge)  # Track the added edge

    # Set the current layer as the previous layer for the next iteration
    previous_layer_nodes = layer_nodes

# Save and view the graph (optional)
dot.render('dfs_transaction_graph1', format='png', cleanup=True)
dot.view()

'dfs_transaction_graph1.pdf'

In [15]:
from graphviz import Digraph

# Function to create individual graphs for each Layer 1 node
def create_individual_graph(layer1_node):
    # Create a directed graph using Graphviz with hierarchical layout
    dot = Digraph()

    # Set the graph layout to be hierarchical (left-right) and use L-shaped edges
    dot.attr(rankdir='LR', splines='ortho')

    # Add the current Layer 1 node
    node_color = 'black'
    
    # Change node color if 'WITHDRAWAL' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
    if 'WITHDRAWAL' in layer1_node['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
        node_color = '#0000ff'  # Blue color for withdrawal
        
    # Change node color if 'ON HOLD' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
    elif 'ON HOLD' in layer1_node['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
        node_color = '#ff0000'  # Red color for on hold

    # Add Layer 1 node with the specific color
    dot.node(
        str(layer1_node['from_account_no']), 
        label=f"Acc: {layer1_node['from_account_no']}\nTransID: {layer1_node['transaction_id']}\nIFSC Code: {layer1_node['ifsc_code']}\nAmount: {format_amount_indian(layer1_node['amount'])}", 
        shape='box',
        color=node_color
    )

    # Initialize the previous layer nodes with the current Layer 1 node
    previous_layer_nodes = pd.DataFrame([layer1_node])

    # Set to track already added edges to avoid duplicates
    added_edges = set()

    # Loop through each distinct layer starting from Layer 2
    for layer in layers[1:]:
        # Step 2: Retrieve records for the current layer
        layer_records = df[df['layer'] == layer]
        layer_nodes = layer_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'ifsc_code', 'amount', 'remarks', 'action_taken_by_bank_wallet_pgpa_merchant_insurance']]

        # Add nodes for the current layer (from_account_no as nodes, with rectangular shape)
        for _, row in layer_nodes.iterrows():
            # Add edges based on "to_account_no" in the current layer matching "from_account_no" in the previous layer
            matching_previous_layer_nodes = previous_layer_nodes[previous_layer_nodes['from_account_no'] == row['to_account_no']]

            for _, prev_row in matching_previous_layer_nodes.iterrows():
                # Add node color logic for the current layer nodes
                node_color = 'black'
                
                if 'WITHDRAWAL' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
                    node_color = '#0000ff'
                elif 'ON HOLD' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
                    node_color = '#ff0000'

                # Add node for the current layer with the specific color
                dot.node(
                    str(row['from_account_no']),
                    label=f"Acc: {row['from_account_no']}\nTransID: {row['transaction_id']}\nIFSC Code: {row['ifsc_code']}\nAmount: {format_amount_indian(row['amount'])}",
                    shape='box',
                    color=node_color
                )

                # Create an identifier for the edge to check for duplicates
                edge = (str(prev_row['from_account_no']), str(row['from_account_no']))

                # Add edge if it hasn't been added yet
                if edge not in added_edges:
                    # Add L-shaped edge from the previous layer's from_account_no to the current layer's from_account_no
                    dot.edge(str(prev_row['from_account_no']), str(row['from_account_no']), label=f"Transfer on {row['transaction_date']}")
                    added_edges.add(edge)  # Track the added edge

        # Set the current layer as the previous layer for the next iteration
        previous_layer_nodes = layer_nodes

    # Save and view the graph for the current node
    filename = f'dfs_transaction_graph_{layer1_node["from_account_no"]}'
    dot.render(filename, format='png', cleanup=True)
    dot.view()

# Get the distinct unique layers from the dataset
layers = sorted(df['layer'].unique())

# Retrieve Layer 1 records where "Money Transfer" occurs
layer1_records = df[(df['layer'] == 1) & (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
layer1_nodes = layer1_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'ifsc_code', 'amount', 'remarks', 'action_taken_by_bank_wallet_pgpa_merchant_insurance']]

# Create separate graphs for each node in Layer 1
for _, layer1_node in layer1_nodes.iterrows():
    create_individual_graph(layer1_node)

In [92]:
from graphviz import Digraph

# Create a directed graph using Graphviz with hierarchical layout
dot = Digraph()

# Set the graph layout to be hierarchical (left-right) and use L-shaped edges
dot.attr(rankdir='LR', splines='ortho')

# Get the distinct unique layers from the dataset
layers = sorted(df['layer'].unique())

# Initialize the previous layer nodes (starting with an empty set)
previous_layer_nodes = None

# Loop through each distinct layer in the dataset
for layer in layers:
    # Step 1: Retrieve records for the current layer where "Money Transfer" occurs
    layer_records = df[(df['layer'] == layer) & (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
    layer_nodes = layer_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'ifsc_code', 'amount', 'remarks']]
    
    # Add nodes for the current layer (from_account_no as nodes, with rectangular shape)
    for _, row in layer_nodes.iterrows():
        dot.node(str(row['from_account_no']), label=f"Acc: {row['from_account_no']}\nTransID: {row['transaction_id']}\nIFSC Code: {row['ifsc_code']}\nAmount: {format_amount_indian(row['amount'])}", shape='box')

    # If this is not the first layer, add edges between the previous layer and the current one
    if previous_layer_nodes is not None:
        # Add edges based on "to_account_no" in the current layer matching "from_account_no" in the previous layer
        for _, row in layer_nodes.iterrows():
            matching_previous_layer_nodes = previous_layer_nodes[previous_layer_nodes['from_account_no'] == row['to_account_no']]
            
            for _, prev_row in matching_previous_layer_nodes.iterrows():
                # Add L-shaped edge from the previous layer's from_account_no to the current layer's from_account_no
                dot.edge(str(prev_row['from_account_no']), str(row['from_account_no']), label=f"Transfer on {row['transaction_date']}")

    # Set the current layer as the previous layer for the next iteration
    previous_layer_nodes = layer_nodes

# Save and view the graph (optional)
dot.render('dfs_transaction_graph', format='png', cleanup=True)
dot.view()

'dfs_transaction_graph.pdf'

In [91]:
from graphviz import Digraph

# Create a directed graph using Graphviz with hierarchical layout
dot = Digraph()

# Set the graph layout to be hierarchical (top-down) and use L-shaped edges
dot.attr(rankdir='LR', splines='ortho')

# Step 1: Retrieve Layer 1 records where "Money Transfer" occurs
layer1_records = df[(df['layer'] == 1) & (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
layer1_nodes = layer1_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date','ifsc_code', 'amount', 'remarks']]

# Add Layer 1 nodes (from_account_no as nodes, with rectangular shape)
for _, row in layer1_nodes.iterrows():
    dot.node(str(row['from_account_no']), label=f"Acc: {row['from_account_no']}\nTransID: {row['transaction_id']}\nIFSC Code: {row['ifsc_code']}\nAmount: {format_amount_indian(row['amount'])}", shape='box')

# Step 2: Retrieve Layer 2 records where "Money Transfer" occurs
layer2_records = df[(df['layer'] == 2) & (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
layer2_nodes = layer2_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date','ifsc_code', 'amount', 'remarks']]

# Add Layer 2 nodes and connect edges based on "to_account_no" in Layer 2 matching "from_account_no" in Layer 1
for _, row in layer2_nodes.iterrows():
    # Find matching Layer 1 nodes where to_account_no equals from_account_no in Layer 2
    matching_layer1_nodes = layer1_nodes[layer1_nodes['from_account_no'] == row['to_account_no']]
    
    for _, layer1_row in matching_layer1_nodes.iterrows():
        # Add Layer 2 node with rectangular shape
        dot.node(str(row['from_account_no']), label=f"Acc: {row['from_account_no']}\nTransID: {row['transaction_id']}\nIFSC Code: {row['ifsc_code']}\nAmount: {format_amount_indian(row['amount'])}", shape='box')
        
        # Add L-shaped edge from Layer 1's from_account_no to Layer 2's from_account_no
        dot.edge(str(layer1_row['from_account_no']), str(row['from_account_no']), label=f"Transfer on {row['transaction_date']}\nAmount: {format_amount_indian(row['amount'])}")

# Step 3: Retrieve Layer 3 records where "Money Transfer" occurs
layer3_records = df[(df['layer'] == 3) & (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
layer3_nodes = layer3_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date','ifsc_code', 'amount', 'remarks']]

# Add Layer 3 nodes and connect edges based on "to_account_no" in Layer 3 matching "from_account_no" in Layer 2
for _, row in layer3_nodes.iterrows():
    # Find matching Layer 2 nodes where to_account_no equals from_account_no in Layer 3
    matching_layer2_nodes = layer2_nodes[layer2_nodes['from_account_no'] == row['to_account_no']]
    
    for _, layer2_row in matching_layer2_nodes.iterrows():
        # Add Layer 3 node with rectangular shape
        dot.node(str(row['from_account_no']), label=f"Acc: {row['from_account_no']}\nTransID: {row['transaction_id']}\nIFSC Code: {row['ifsc_code']}\nAmount: {format_amount_indian(row['amount'])}", shape='box')
        
        # Add L-shaped edge from Layer 2's from_account_no to Layer 3's from_account_no
        dot.edge(str(layer2_row['from_account_no']), str(row['from_account_no']), label=f"Transfer on {row['transaction_date']}\nAmount: {format_amount_indian(row['amount'])}")

# Save and view the graph (optional)
dot.render('dfs_transaction_graph', format='png', cleanup=True)
dot.view()

'dfs_transaction_graph.pdf'

In [13]:
# Step 1: Retrieve Layer 1 records where "Money Transfer" occurs
layer1_records = df[(df['layer'] == 1) & (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
layer1_nodes = layer1_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'amount', 'remarks']]

# Create a directed graph
G = nx.DiGraph()

# Add Layer 1 nodes (from_account_no as nodes, with details as attributes)
for _, row in layer1_nodes.iterrows():
    G.add_node(row['from_account_no'], 
               to_account=row['to_account_no'], 
               transaction_id=row['transaction_id'], 
               transaction_date=row['transaction_date'], 
               amount= format_amount_indian(row['amount']), 
               remarks=row['remarks'])

# Step 2: Retrieve Layer 2 records where "Money Transfer" occurs
layer2_records = df[(df['layer'] == 2) & (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
layer2_nodes = layer2_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'amount', 'remarks']]

# Add Layer 2 nodes and connect edges based on "to_account_no" in Layer 2 matching "from_account_no" in Layer 1
for _, row in layer2_nodes.iterrows():
    # Find matching Layer 1 nodes where to_account_no equals from_account_no in Layer 2
    matching_layer1_nodes = layer1_nodes[layer1_nodes['from_account_no'] == row['to_account_no']]
    
    for _, layer1_row in matching_layer1_nodes.iterrows():
        # Add Layer 2 node
        G.add_node(row['from_account_no'], 
                   to_account=row['to_account_no'], 
                   transaction_id=row['transaction_id'], 
                   transaction_date=row['transaction_date'], 
                   amount= format_amount_indian(row['amount']),
                   remarks=row['remarks'])
        
        # Add edges from Layer 1's from_account_no to Layer 2's from_account_no
        G.add_edge(layer1_row['from_account_no'], row['from_account_no'])

# Step 3: Retrieve Layer 3 records where "Money Transfer" occurs
layer3_records = df[(df['layer'] == 3) & (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
layer3_nodes = layer3_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'amount', 'remarks']]

# Add Layer 3 nodes and connect edges based on "to_account_no" in Layer 3 matching "from_account_no" in Layer 2
for _, row in layer3_nodes.iterrows():
    # Find matching Layer 1 nodes where to_account_no equals from_account_no in Layer 2
    matching_layer2_nodes = layer2_nodes[layer2_nodes['from_account_no'] == row['to_account_no']]
    
    for _, layer2_row in matching_layer2_nodes.iterrows():
        # Add Layer 3 node
        G.add_node(row['from_account_no'], 
                to_account=row['to_account_no'], 
                transaction_id=row['transaction_id'], 
                transaction_date=row['transaction_date'], 
                amount= format_amount_indian(row['amount']),
                remarks=row['remarks'])
        
        # Add edges from Layer 1's from_account_no to Layer 2's from_account_no
        G.add_edge(layer2_row['from_account_no'], row['from_account_no'])


In [15]:
# Create a directed graph
G = nx.DiGraph()

# Iterate over the layers from 1 to 7
for layer in range(1, 8):
    # Retrieve records for the current layer where "Money Transfer" occurs
    layer_records = df[(df['layer'] == layer) & 
                       (df['action_taken_by_bank_wallet_pgpa_merchant_insurance'].str.contains('Money Transfer'))]
    
    # Extract relevant columns
    layer_nodes = layer_records[['from_account_no', 'to_account_no', 'transaction_id', 
                                  'transaction_date', 'amount', 'remarks']]
    
    # Process each record in the current layer
    for _, row in layer_nodes.iterrows():
        # Add the current layer node with 'layer' attribute
        G.add_node(row['from_account_no'], 
                   layer=layer,  # Add 'layer' attribute
                   to_account=row['to_account_no'], 
                   transaction_id=row['transaction_id'], 
                   transaction_date=row['transaction_date'], 
                   amount=format_amount_indian(row['amount']), 
                   remarks=row['remarks'])
        
        # If this is not the first layer, connect edges to the previous layer's nodes
        if layer > 1:
            # Get nodes from the previous layer
            prev_layer_nodes = df[(df['layer'] == layer - 1) & 
                                  (df['to_account_no'] == row['from_account_no'])]
            
            for _, prev_row in prev_layer_nodes.iterrows():
                # Add edge from previous layer's from_account_no to current layer's from_account_no
                G.add_edge(prev_row['from_account_no'], row['from_account_no'])

In [16]:
# Create elements for Dash Cytoscape
cyto_elements = []

# Custom positions to create an L-shape for edges
layer_xs = [100, 300, 600, 900, 1200, 1500, 1800, 2100]  # x-coordinates for each layer
y_increment = 100
current_ys = [100] * 8  # y-coordinates for each layer

# Create a dictionary to store node positions for each layer
layer_node_positions = {}

# Iterate over the layers from 0 to 7
for layer in range(8):
    layer_node_positions[layer] = {}  # Initialize node positions for the current layer
    
    # Iterate over the nodes in the current layer
    for node in G.nodes(data=True):
        if node[1]['layer'] == layer:  # Node belongs to the current layer
            # Define the content string with details in multi-line format
            content = (f"{node[0]}\n"
                       f"To: {node[1]['to_account']}\n"
                       f"Transaction ID: {node[1]['transaction_id']}\n"
                       f"Date: {node[1]['transaction_date']}\n"
                       f"Amount: {node[1]['amount']}\n"
                       f"Remarks: {node[1]['remarks']}")
            
            # Add the node to cyto_elements
            cyto_elements.append({
                'data': {'id': node[0], 'label': content},  # Display node content
                'position': {'x': layer_xs[layer], 'y': current_ys[layer]}  # Positioning for the current layer
            })
            layer_node_positions[layer][node[0]] = (layer_xs[layer], current_ys[layer])  # Track node position for edges
            current_ys[layer] += y_increment  # Increment y-coordinate for the next node in the same layer

# Add edges between nodes in adjacent layers
for edge in G.edges():
    source_layer = next((layer for layer, nodes in enumerate(G.nodes(data=True)) if edge[0] in [node[0] for node in nodes]), None)
    target_layer = next((layer for layer, nodes in enumerate(G.nodes(data=True)) if edge[1] in [node[0] for node in nodes]), None)
    
    if source_layer is not None and target_layer is not None:
        cyto_elements.append({
            'data': {'source': edge[0], 'target': edge[1]},
            'style': {'line-color': '#ff0000', 'width': 2, 'target-arrow-shape': 'triangle', 'arrow-color': '#0074D9'}
        })

In [ ]:

# Create elements for Dash Cytoscape
cyto_elements = []

# Custom positions to create an L-shape for edges
layer1_x = 100
layer2_x = 300  # Shift layer 2 nodes further right
layer3_x = 600
y_increment = 100
current_y = 100

# Add nodes for Layer 1
layer1_node_positions = {}  # Keep track of node positions
for i, node in enumerate(G.nodes(data=True)):
    if node[1]['to_account'] in layer1_nodes['to_account_no'].values:  # Layer 1 nodes
        # Define the content string with details in multi-line format
        content = (f"{node[0]}\n"
                   f"To: {node[1]['to_account']}\n"
                   f"Transaction ID: {node[1]['transaction_id']}\n"
                   f"Date: {node[1]['transaction_date']}\n"
                   f"Amount: {node[1]['amount']}\n"
                   f"Remarks: {node[1]['remarks']}")
         
        cyto_elements.append({
            'data': {'id': node[0], 'label': content},  # Display node content
            'position': {'x': layer1_x, 'y': current_y}  # Positioning for layer 1
        })
        layer1_node_positions[node[0]] = (layer1_x, current_y)  # Track Layer 1 node position for edges
        current_y += y_increment

# Reset y position for Layer 2
current_y = 100

# Add nodes for Layer 2 and edges
layer2_node_positions = {}  # Keep track of layer 2 node positions
for node in G.nodes(data=True):
    if node[1]['to_account'] not in layer1_nodes['to_account_no'].values:  # Layer 2 nodes
        # Define the content string with details in multi-line format
        content = (f"{node[0]}\n"
                   f"To: {node[1]['to_account']}\n"
                   f"Transaction ID: {node[1]['transaction_id']}\n"
                   f"Date: {node[1]['transaction_date']}\n"
                   f"Amount: {node[1]['amount']}\n"
                   f"Remarks: {node[1]['remarks']}")
        
        cyto_elements.append({
            'data': {'id': node[0], 'label': content},  # Display node content
            'position': {'x': layer2_x, 'y': current_y}  # Positioning for layer 2
        })
        layer2_node_positions[node[0]] = (layer2_x, current_y)  # Track Layer 2 node position for edges
        current_y += y_increment

current_y = 100
# Add nodes for Layer 2 and edges
layer3_node_positions = {}  # Keep track of layer 2 node positions
for node in G.nodes(data=True):
    if node[1]['to_account'] not in layer2_nodes['to_account_no'].values:  # Layer 2 nodes
        # Define the content string with details in multi-line format
        content = (f"{node[0]}\n"
                   f"To: {node[1]['to_account']}\n"
                   f"Transaction ID: {node[1]['transaction_id']}\n"
                   f"Date: {node[1]['transaction_date']}\n"
                   f"Amount: {node[1]['amount']}\n"
                   f"Remarks: {node[1]['remarks']}")
        
        cyto_elements.append({
            'data': {'id': node[0], 'label': content},  # Display node content
            'position': {'x': layer3_x, 'y': current_y}  # Positioning for layer 2
        })
        layer3_node_positions[node[0]] = (layer3_x, current_y)  # Track Layer 2 node position for edges
        current_y += y_increment

# Add edges between Layer 1 and Layer 2
for edge in G.edges():
    cyto_elements.append({
        'data': {'source': edge[0], 'target': edge[1]},
        'style': {'line-color': '#ff0000', 'width': 2, 'target-arrow-shape': 'triangle', 'arrow-color': '#0074D9'}
    })

# Initialize Dash app
app = JupyterDash(__name__)

app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape-network',
        elements=cyto_elements,
        style={'width': '100%', 'height': '800px'},
        layout={'name': 'breadthfirst'},  # Use preset layout to keep positions defined
        stylesheet=[
            {'selector': 'node', 'style': {'content': 'data(label)', 'shape': 'rectangle', 'background-color': '#00ffD9', 'text-wrap': 'wrap', 'font-size': '12px', 'text-valign': 'bottom', 'text-halign': 'right'}},
            {'selector': 'edge', 'style': {'line-color': '#ff0000', 'curve-style': 'straight', 'target-arrow-shape': 'triangle'}}  # Straight edge styling
        ]
    )
])

if __name__ == '__main__':
    app.run_server(debug=True, port=4282)